## Week 6 : Large Language Models
```
- Generative Artificial Intelligence (Fall semester 2023)
- Professor: Muhammad Fahim
- Teaching Assistant: Ahmad Taha
```
<hr>

## Contents
```
1. Transformers (Implementing a transformer)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding

```

<hr>


# Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](https://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)


In [1]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np

from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda' and torch.cuda.get_device_capability() == (8, 9):
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True

### Transformer Encoder with Pytorch

In [2]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=32)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

C:\Users\sasha\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [3]:
encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
**The main goal is to efficiently encode the data**

![](https://jalammar.github.io/images/t/encoder_with_tensors.png)

## Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They’re abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](https://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [4]:
# simple sequence = I am here today
simple_sequence_embedding = torch.rand((4, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))

# Create key, query and value for each word in the sentence
queries = simple_sequence_embedding.mm(Wq) # self.q(embedding[0])
keys = simple_sequence_embedding.mm(Wk)
values = simple_sequence_embedding.mm(Wv)

In [5]:
values

tensor([[-3.2226,  0.7849, -2.7105, -0.5892, -3.3174, -0.1117, -1.3133],
        [-3.1727,  2.7164, -5.2874,  2.4067,  0.7304, -0.3600, -4.0256],
        [-1.3231,  1.1102, -2.7247,  1.6277, -0.6082,  4.3334, -3.2851],
        [-2.3367,  3.2619, -3.2394,  1.8764, -0.0952, -1.9163, -0.1464]])

In [6]:
simple_sequence_embedding

tensor([[0.3880, 0.4886, 0.1934,  ..., 0.0137, 0.1588, 0.1312],
        [0.3019, 0.6626, 0.0480,  ..., 0.2764, 0.1893, 0.5873],
        [0.9736, 0.0812, 0.3543,  ..., 0.6544, 0.8385, 0.9371],
        [0.8726, 0.7077, 0.0695,  ..., 0.3798, 0.7192, 0.7933]])

## Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [7]:
scores = torch.mm(queries, keys.T)
scores

tensor([[ 40.5782,   2.2623,  21.9943,  -1.2945],
        [ 51.6150,  10.9989,  30.9627,  10.3982],
        [ 50.4158,  -1.4186,  32.1361,   5.0784],
        [ 24.7262, -11.5408,  17.9812,  -2.8739]])

## Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [8]:
scores = scores / 8
scores

tensor([[ 5.0723,  0.2828,  2.7493, -0.1618],
        [ 6.4519,  1.3749,  3.8703,  1.2998],
        [ 6.3020, -0.1773,  4.0170,  0.6348],
        [ 3.0908, -1.4426,  2.2477, -0.3592]])

## Step 4

Pass the result through a softmax operation

In [9]:
scores = torch.nn.functional.softmax(scores, dim=1)
scores

tensor([[0.8996, 0.0075, 0.0881, 0.0048],
        [0.9194, 0.0057, 0.0696, 0.0053],
        [0.9035, 0.0014, 0.0920, 0.0031],
        [0.6790, 0.0073, 0.2922, 0.0216]])

## Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [10]:
scores.shape, values.shape

(torch.Size([4, 4]), torch.Size([4, 7]))

In [11]:
z = scores @ values
z

tensor([[-3.0506,  0.8399, -2.7336, -0.3596, -3.0329,  0.2696, -1.5018],
        [-3.0855,  0.8318, -2.7291, -0.4047, -3.0886,  0.1865, -1.4598],
        [-3.0451,  0.8253, -2.7171, -0.3735, -3.0526,  0.2911, -1.4948],
        [-2.6481,  0.9475, -2.7449,  0.1336, -2.4268,  1.1464, -1.8841]])

# Multi-headed attention

**GOAL**:
1. Expand the model’s ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrics down into a single matrix by concatenating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [12]:
from torch import Tensor
import torch.nn.functional as f


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

## Now lets make attention head

In [13]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        # Fill in the missing parts of the constructor to initialize the query, key, and value linear transformations
        # dim_in is the input dimension, dim_q and dim_k are the output dimensions for the queries and keys/values respectively
        # Example: dim_in 512, dim_q = dim_k 64 in the paper
        self.linear_q = nn.Linear(dim_in, dim_q)
        self.linear_k = nn.Linear(dim_in, dim_k)
        self.linear_v = nn.Linear(dim_in, dim_k)

    def forward(self, query, key, value):
        # Implement the forward pass by calling scaled_dot_product_attention function
        # You need to transform the query, key, and value using the linear transformations defined in __init__
        # Fill in with the correct method calls
        query = self.linear_q(query)
        key = self.linear_k(key)
        value = self.linear_v(value)
        
        return scaled_dot_product_attention(query, key, value)

## Multi Head Attention

In [14]:
class MultiHeadToAttention(nn.Module):
    def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
        super().__init__()
        # Initialize heads as multi-AttentionHead instances
        # Initialize linear to combine the outputs of all heads into a single output
        self.heads = nn.ModuleList([AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)])
        self.linear = nn.Linear(number_of_heads * dim_k, dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor):
        # Concatenate outputs from all heads and apply the final linear transformation
        concat_heads = torch.cat([head(query, key, value) for head in self.heads], dim=-1)
        return self.linear(concat_heads)

## Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*C3a9RL6-SFC6fW8NGpJg5A.png)

In [15]:
def position_encoding(seq_len, dim_model, device):
    # Define the position tensor 'pos' with dimensions appropriate for sequence length
    pos = torch.arange(seq_len, dtype=torch.float).unsqueeze(1).to(device)

    # Define the dimension tensor 'dim' suitable for the model dimensions
    dim = torch.arange(0, dim_model, step=2, dtype=torch.float).to(device)

    # Calculate the phase using the position and dimension tensors
    phase = pos / (10000 ** (dim / dim_model))

    # Return the sinusoidal position encoding
    # Complete this line to select sin or cos based on even/odd index
    return torch.cat([torch.sin(phase), torch.cos(phase)], dim=1)

## Encoder Feed Forward

In [16]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
    return nn.Sequential(nn.Linear(dim_input, dim_feedforward),
                         nn.ReLU(),
                         nn.Linear(dim_feedforward, dim_input)
                         )

## Encoder Residual

From the original paper the author implementation

In [17]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

## Putting all together on encoder side

![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## Putting the Encoder layer together

In [18]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Define dimensions for query and key based on model dimension and number of heads
        
        # Initialize the MultiHeadAttention component with a residual connection and dropout
        self.attention = Residual(MultiHeadToAttention(num_heads, dim_model, dim_model // num_heads, dim_model // num_heads), dim_model, dropout)

        # Initialize the feedforward component with a residual connection and dropout
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dim_model, dropout)

    def forward(self, src):
        # Apply the attention mechanism to the source data
        src = self.attention(src, src, src)

        # Apply the feedforward network to the output of the attention mechanism
        return self.feed_forward(src)

## Putting together transformer Encoder part

In [19]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=12, dim_model=512, num_heads=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Initialize a list of TransformerEncoderLayer instances
        self.layers = nn.ModuleList([TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers)])

    def forward(self, src):
        # Retrieve the sequence length and dimension from the source tensor
        seq_len, dimension = src.shape[1], src.shape[2]

        # Add position encoding to the source tensor
        src += position_encoding(seq_len, dimension, device)

        # Process each layer in the transformer encoder
        for layer in self.layers:
            src = layer(src)
        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.


![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

## Decoder layer

The “Encoder-Decoder Attention” layer works just like multi-headed self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

**Task**: implement the decoder layer

In [20]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        dim_model: int = 128,
        num_heads: int = 6,
        dim_feedforward: int = 512,
        dropout: float = 0.1,
    ):
        super().__init__()
        # init dim_q and dim_k as in the encoder
        # Initialize the first self-attention layer with a residual connection
        self.attention_1 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_model // num_heads, dim_model // num_heads), dim_model, dropout)

        # Initialize the second attention layer for interaction with the encoder output
        self.attention_2 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_model // num_heads, dim_model // num_heads), dim_model, dropout)

        # Initialize the feed-forward network
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dim_model, dropout)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Self-attention mechanism
        tgt = self.attention_1(tgt, tgt, tgt)

        # Cross-attention mechanism where the decoder attends to the encoder's output
        tgt = self.attention_2(tgt, memory, memory)

        # Pass through the feed-forward network
        return self.feed_forward(tgt)

## Full Transformer Decoder

**Task**: implement the transformer decoder part class

In [21]:
class TransformerDecoder(nn.Module):
    def __init__(
        self,
        num_layers: int = 6,
        dim_model: int = 128,
        num_heads: int = 8,
        dim_feedforward: int = 512,
        dropout: float = 0.1
    ):
        super().__init__()
        # Initialize layers from TransformerDecoderLayer instances
        self.layers = nn.ModuleList([TransformerDecoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers)])
        # Define the output linear transformation
        self.linear = feed_forward(dim_model, dim_feedforward)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Calculate sequence length and dimension from the target tensor
        seq_len, dimension = tgt.shape[1], tgt.shape[2]

        # Add position encoding to the target tensor
        tgt += position_encoding(seq_len, dimension, device)

        # Process each layer in the transformer decoder
        # Code here
        for layer in self.layers:
            tgt = layer(tgt, memory)

        # Apply a softmax to the output of the final linear layer
        tgt = self.linear(tgt)
        tgt = f.softmax(tgt, dim=-1)
        return tgt

## Full Transformer model

**Task**:
1. Assembly a full transformer (Encoder + Decoder)
2. Implement the Transformer training loop
3. Using dataset of your choice train the transformer just for one epoch


In [22]:
import datasets
import torchtext

In [23]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

## Get basic english tokenizer
tokenizer = torchtext.data.utils.get_tokenizer("basic_english")

def tokenize_sample(sample, tokenizer, max_length=200):
    tokens = tokenizer(sample["text"])[:max_length]
    length = len(tokens)
    return {"tokens": tokens, "length": length}

In [24]:
max_text_length = 256

train_data = train_data.map(
    tokenize_sample, fn_kwargs={"tokenizer": tokenizer, "max_length": max_text_length}
)
test_data = test_data.map(
    tokenize_sample, fn_kwargs={"tokenizer": tokenizer, "max_length": max_text_length}
)

In [25]:
min_freq = 5
special_tokens = ["<unk>", "<pad>"]

vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

unk_index = vocab["<unk>"]
pad_index = vocab["<pad>"]
vocab.set_default_index(unk_index)

In [26]:
def encode_sample(example, vocab):
    ids = vocab.lookup_indices(example["tokens"])
    return {"ids": ids}

train_data = train_data.map(encode_sample, fn_kwargs={"vocab": vocab})
test_data = test_data.map(encode_sample, fn_kwargs={"vocab": vocab})


train_data = train_data.with_format(type="torch", columns=["ids", "label", "length"])
test_data = test_data.with_format(type="torch", columns=["ids", "label", "length"])

In [27]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_ids = [i["ids"] for i in batch]
        batch_ids = nn.utils.rnn.pad_sequence(
            batch_ids, padding_value=pad_index, batch_first=True
        )
        batch_length = [i["length"] for i in batch]
        batch_length = torch.stack(batch_length)
        batch_label = [i["label"] for i in batch]
        batch_label = torch.stack(batch_label)
        batch = {"ids": batch_ids, "length": batch_length, "label": batch_label}
        return batch

    return collate_fn

def create_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader


batch_size = 64

train_data_loader = create_data_loader(train_data, batch_size, pad_index, shuffle=True)
test_data_loader = create_data_loader(test_data, batch_size, pad_index)

In [28]:
class Transformer(nn.Module):
    def __init__(
            self,
            num_layers: int = 6,
            dim_model: int = 128,
            num_heads: int = 8,
            dim_feedforward: int = 512,
            dropout: float = 0.1,
    ):
        super().__init__()
        self.encoder = TransformerEncoder(num_layers, dim_model, num_heads, dim_feedforward, dropout)
        self.decoder = TransformerDecoder(num_layers, dim_model, num_heads, dim_feedforward, dropout)

    def forward(self, src, tgt):
        memory = self.encoder(src)
        decoder_output = self.decoder(tgt, memory)
        return decoder_output

In [29]:
class TransformerNet(nn.Module):
    def __init__(
            self,
            pad_index,
            vocab_size,
            num_layers: int = 8,
            dim_model: int = 128,
            num_heads: int = 8,
            dim_feedforward: int = 512,
            dropout: float = 0.1,
            dim_target: int = 10,
    ):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, dim_model, padding_idx=pad_index)
        self.transformer = Transformer(num_layers, dim_model, num_heads, dim_feedforward, dropout)
        self.linear = nn.Linear(dim_model * max_text_length, dim_target)
        
    def forward(self, src):
        src = self.embedding(src)
        output = self.transformer(src, src).view(batch_size, -1)
        return self.linear(output)

In [30]:
vocab_size = len(vocab)
output_dim = len(train_data.unique("label"))
lr = 5e-4

model = TransformerNet(pad_index, vocab_size, dim_target=output_dim).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [31]:
def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [32]:
n_epochs = 1
for ep in range(n_epochs):
    model.train()
    epoch_losses = []
    epoch_accs = []
    for batch in train_data_loader:
        optimizer.zero_grad()
        ids = batch["ids"].to(device)
        label = batch["label"].to(device)

        prediction = model(ids)
        loss = criterion(prediction, label)
        accuracy = get_accuracy(prediction, label)

        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())
    print(f'[Epoch {ep}] Loss: {np.mean(epoch_losses)}, Acc: {np.mean(epoch_accs)}')

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [16384, 128]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).